# Đề thi Giữa kỳ Hệ khuyến nghị 2025.

### Khoa học dữ liệu IUH. Ngày 16/10/2025. Thời gian làm bài: 90 phút. Ca 1.

**Bài 1.** (*2 điểm*) Cho file dữ liệu *data.csv* về $25$ bài hát được $400$ users đã rate trong tháng qua, điểm từ $1$ đến $5$. Cấu trúc file: *id user, id bài hát, điểm rating*. Gọi $a$ là tổng hai chữ số cuối trong MSSV của anh/chị.

1) Sử dụng kỹ thuật tổng hợp 2 thông tin: số rating kết hợp rating trung bình, hãy đề xuất ra top $10$ bài hát nên nghe cho một user cold-start.
2) Giả sử có một user đã nghe bài hát STT $a$. Sử dụng kỹ thuật đếm số cặp bài hát được nghe bởi cùng user, hãy đề xuất ra top 5 bài hát mà user này có thể cũng muốn nghe theo xu hướng.

### 1.1

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv', names=['userid', 'songid', 'rating'])
print("Dữ liệu rating:")
print(f"Số users: {df['userid'].nunique()}")
print(f"Số songs: {df['songid'].nunique()}")
print(f"Số ratings: {len(df)}")
display(df.head(5))

Dữ liệu rating:
Số users: 368
Số songs: 25
Số ratings: 1679


,userid,songid,rating
0,1,18,5
1,3,25,4
2,3,9,3
3,3,15,1
4,3,21,2


In [8]:
song_stats = df.groupby('songid').agg(num_ratings=('rating', 'count'),
                                      avg_rating =('rating', 'mean')).reset_index()

# Tham số toàn cục
C = song_stats['avg_rating'].mean()
m = song_stats['num_ratings'].quantile(0.5)

# Thông tin
print("Rating trung bình: ", C)
print("Giá trị num_ratings ở quantile 0.5: ", m)
print("Kích thước dữ liệu ở q = 0.5:", song_stats[song_stats['num_ratings'] >= m].shape)
print("Kích thước dữ liệu ban đầu:", song_stats.shape)


q_movies = song_stats[song_stats['num_ratings'] >= m].copy()

def weighted_rating_row(x, m=m, C=C):
    v, R = x['num_ratings'], x['avg_rating']
    return (v/(v+m))*R + (m/(v+m))*C

q_movies['score'] = q_movies.apply(weighted_rating_row, axis=1)
top5 = q_movies.sort_values('score', ascending=False).head(10)
display(top5)

Rating trung bình:  2.9718108401981596
Giá trị num_ratings ở quantile 0.5:  67.0
Kích thước dữ liệu ở q = 0.5: (13, 3)
Kích thước dữ liệu ban đầu: (25, 3)


,songid,num_ratings,avg_rating,score
3,4,70,3.400000,3.190594
2,3,69,3.347826,3.162583
0,1,70,3.100000,3.037309
24,25,71,3.098592,3.037039
6,7,71,3.084507,3.029792
11,12,67,3.044776,3.008293
12,13,85,2.952941,2.961259
15,16,79,2.924051,2.945968
22,23,74,2.905405,2.936960
4,5,74,2.864865,2.915683


### 1.2

In [39]:
# mssv = 23644831
a = 3 + 1
users_listened_song = df.loc[df['songid'] == a, 'userid'].unique()
co_listened = df[df['userid'].isin(users_listened_song)]
co_listened = co_listened[co_listened['songid'] != a]
co_occurrence = (
    co_listened.groupby('songid')
    .size()
    .reset_index(name='pair_count')
    .sort_values(['pair_count', 'songid'], ascending=[False, True])
)
top_5_song1 = co_occurrence.head(5).merge(song_stats, on='songid', how='left')
print('\nTop 5 bài hát thường được nghe cùng bài hát 4:')
display(top_5_song1[['songid', 'pair_count', 'avg_rating']])


Top 5 bài hát thường được nghe cùng bài hát 4:


,songid,pair_count,avg_rating
0,13,20,2.952941
1,9,18,2.805556
2,17,17,2.544118
3,16,16,2.924051
4,23,16,2.905405


**Bài 2.** (*2.5 điểm*) Sử dụng dataset movie_lens trong đó có file *movies.csv* mô tả tên films và các thể loại (một phim có thể thuộc nhiều thể loại).

1. Đọc dữ liệu và lọc lại $200$ films có nhiều thể loại nhất. Ngoài ra, hãy cho biết tổng cộng có bao nhiêu thể loại khác nhau.
2. Xét film $X$ ở vị trí $25$ trong danh sách nêu trên, tính ra top $10$ films tương đồng với film $X$ này nhất bằng *độ tương đồng Jaccard*.
3. Một user $Y$ đã xem các bộ phim có STT trong tập hợp $ID$, hãy xây dựng profile cơ bản cho user này và khuyến nghị ra top $10$ films mà user $Y$ này có thể muốn xem.

### 2.1

In [40]:
#CODE HERE
movie_df = pd.read_csv("movies.csv")
movie_df['genres'] = movie_df['genres'].str.split('|')
movie_df["count_genres"] = movie_df["genres"].apply(len)
# 1. Lọc 200 films có nhiều thể loại nhất
top_200_movies = movie_df.sort_values('count_genres', ascending=False).head(200).copy()
print(f"Top 200 movies with most genres:")
display(top_200_movies[['title', 'genres', 'count_genres']].head())

Top 200 movies with most genres:


,title,genres,count_genres
7441,Rubber (2010),"[Action, Adventure, Comedy, Crime, Drama, Film...",10
5556,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Action, Animation, Crime, Drama, Film-Noir, M...",8
6626,Enchanted (2007),"[Adventure, Animation, Children, Comedy, Fanta...",7
2250,Who Framed Roger Rabbit? (1988),"[Adventure, Animation, Children, Comedy, Crime...",7
5819,Robots (2005),"[Adventure, Animation, Children, Comedy, Fanta...",7


In [58]:
list_genre = []
for i in movie_df["genres"]:
    list_genre.extend(i)

display(list_genre[:10])
display("So luong the loai:", len(pd.DataFrame(list_genre).value_counts()))
display(pd.DataFrame(list_genre).value_counts())


['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Adventure',
 'Children',
 'Fantasy',
 'Comedy',
 'Romance']

'So luong the loai:'

20

Drama                 4361
Comedy                3756
Thriller              1894
Action                1828
Romance               1596
Adventure             1263
Crime                 1199
Sci-Fi                 980
Horror                 978
Fantasy                779
Children               664
Animation              611
Mystery                573
Documentary            440
War                    382
Musical                334
Western                167
IMAX                   158
Film-Noir               87
(no genres listed)      34
dtype: int64

### 2.2

In [15]:
# 2. Hàm tính độ tương đồng Jaccard
def jaccard_similarity(set1, set2):
    """Tính độ tương đồng Jaccard giữa hai tập hợp"""
    set1 = set(set1)
    set2 = set(set2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# 3. Chọn một film X (ví dụ: film đầu tiên trong top 100)
film_x_index = 25
film_x = top_200_movies.iloc[film_x_index]
film_x_genres = set(film_x['genres'])

print(f"\nFilm X được chọn: {film_x['title']}")
print(f"Genres của Film X: {film_x['genres']}")

# 4. Tính độ tương đồng Jaccard với tất cả các films khác
similarities = []
for idx, row in top_200_movies.iterrows():
    if idx != top_200_movies.iloc[film_x_index].name:  # Bỏ qua chính film X
        similarity = jaccard_similarity(film_x_genres, row['genres'])
        similarities.append({
            'movieId': row['movieId'],
            'title': row['title'],
            'genres': row['genres'],
            'jaccard_similarity': similarity
        })

# 5. Chuyển thành DataFrame và sắp xếp theo độ tương đồng
similarity_df = pd.DataFrame(similarities)
similarity_df = similarity_df.sort_values('jaccard_similarity', ascending=False)

# 6. Hiển thị top 10 films tương đồng nhất
print(f"\nTop 10 films tương đồng nhất với '{film_x['title']}':")
top_10_similar = similarity_df.head(10)
display(top_10_similar[['title', 'genres', 'jaccard_similarity']])


Film X được chọn: Madagascar: Escape 2 Africa (2008)
Genres của Film X: ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'IMAX']

Top 10 films tương đồng nhất với 'Madagascar: Escape 2 Africa (2008)':


,title,genres,jaccard_similarity
46,Kung Fu Panda 2 (2011),"[Action, Adventure, Animation, Children, Comed...",1.000000
10,Mars Needs Moms (2011),"[Action, Adventure, Animation, Children, Comed...",0.857143
113,Zootopia (2016),"[Action, Adventure, Animation, Children, Comedy]",0.833333
185,"Incredibles, The (2004)","[Action, Adventure, Animation, Children, Comedy]",0.833333
156,Kung Fu Panda (2008),"[Action, Animation, Children, Comedy, IMAX]",0.833333
180,Bolt (2008),"[Action, Adventure, Animation, Children, Comedy]",0.833333
69,Toy Story 3 (2010),"[Adventure, Animation, Children, Comedy, Fanta...",0.714286
33,Megamind (2010),"[Action, Animation, Children, Comedy, Sci-Fi, ...",0.714286
37,TMNT (Teenage Mutant Ninja Turtles) (2007),"[Action, Adventure, Animation, Children, Comed...",0.714286
38,Meet the Robinsons (2007),"[Action, Adventure, Animation, Children, Comed...",0.714286


### 2.3

In [61]:
watched_idx = [2, 3, 6, 4,4, 8,3,1]
watched_genres = set()
for i in watched_idx:
    watched_genres |= set(top_200_movies.iloc[i]["genres"])

rec_scores = []
for idx, row in top_200_movies.iterrows():
    if idx in [top_200_movies.index[i] for i in watched_idx]:
        continue
    score = jaccard_similarity(watched_genres, row["genres"])
    rec_scores.append((row["movieId"], row["title"], score))

rec_df = pd.DataFrame(rec_scores, columns=["movieId", "title", "score"]).sort_values("score", ascending=False)
user_top10 = rec_df.head(10)
print("\nUser Y genre profile:", sorted(list(watched_genres)))
print("Top-10 films recommended to user Y (content-based by genres):")
print(user_top10)


User Y genre profile: ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']
Top-10 films recommended to user Y (content-based by genres):
    movieId                                              title     score
0     81132                                      Rubber (2010)  0.529412
2      4719                               Osmosis Jones (2001)  0.437500
3      1907                                       Mulan (1998)  0.437500
4     85261                             Mars Needs Moms (2011)  0.437500
5     52462  Aqua Teen Hunger Force Colon Movie Film for Th...  0.437500
6     71999          Aelita: The Queen of Mars (Aelita) (1924)  0.437500
7     79132                                   Inception (2010)  0.437500
1     81847                                     Tangled (2010)  0.437500
47    31921                Seven-Per-Cent Solution, The (1976)  0.375000
53      595

**Bài 3.** (*2.5 điểm*) Cho một bảng *utility matrix* giữa 8 user và 5 item như bên dưới:

|Rating     | $I_1$    | $I_2$  | $I_3$   |  $I_4$  | $I_5$  |
|----------|----------|--------|---------|---------|---------|
| $U_1$| 4    | 3      | 4    |   ?  |  5|
| $U_2$| 2    | 3      | 1    |   5  |  3|
| $U_3$| 5    | 4      |  3  |    4  |  2| 
| $U_4$|   4  |   2    | 2    |   5  |  4|
| $U_5$|   3  |   4    | ?    |   3  |  5|
| $U_6$|   5  |   3    | 5    |   5  |  1|
| $U_7$|   5  |   5    | 5    |   4  |  5|
| $U_8$|   1  |   5    | 2    |   2  |  4|

1. Dự đoán hai missing rating $r_{U_1, I_4}$ và $r_{U_5, I_2}$ bằng cách dùng *độ tương đồng Pearson*.
2. Câu hỏi tương tự trên nhưng chuẩn hoá bảng theo dạng *center-mean* và sau đó dùng *độ tương đồng cosine*.
3. Một người thực hiện dự đoán theo kiểu tuần tự, tức là dự đoán xong missing rating thứ $1$ thì điền rating đó vào bảng hiện tại rồi mới thực hiện tiếp cho missing rating thứ $2$. Anh chị hãy thực hiện cách làm này cho một trong hai ý 1) hoặc 2) và so sánh kết quả thu được. Hỏi cách làm này nói chung sẽ có các ưu nhược điểm gì?

### 3.1

In [74]:
#CODE HERE

from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

# Tạo utility matrix 
utility_matrix = np.array([
    [4, 3, 4, np.nan, 5],  # U1
    [2, 3, 1, 5,      3],       # U2  
    [5, 4, 3, 4,      2],       # U3
    [4, 2, 2, 5,      4],
    [3, 4, np.nan, 3, 5],
    [5, 3, 5, 5,      1],
    [5, 5, 5, 4,      5],
    [1, 5, 2, 2,      4],
])

users = ['U1', 'U2', 'U3', 'U4', "U5", "U6", "U7", "U8"]
items = ['I1', 'I2', 'I3', 'I4', 'I5']

df = pd.DataFrame(utility_matrix, index=users, columns=items)
print("Utility Matrix:")
display(df)

# 1. Dự đoán missing ratings bằng Pearson correlation

def pearson_similarity(user1, user2):
    """Tính độ tương đồng Pearson giữa 2 users"""
    # Tìm các items mà cả 2 users đều đã rating
    common_items = ~(np.isnan(user1) | np.isnan(user2))
    
    if np.sum(common_items) < 2:
        return 0
    
    user1_common = user1[common_items]
    user2_common = user2[common_items]
    
    if np.std(user1_common) == 0 or np.std(user2_common) == 0:
        return 0
    
    correlation, _ = pearsonr(user1_common, user2_common)
    return correlation if not np.isnan(correlation) else 0

def predict_rating_pearson(user_idx, item_idx, matrix):
    """Dự đoán rating cho user và item bằng Pearson correlation"""
    target_user = matrix[user_idx]
    similarities = []
    ratings = []
    
    for i in range(len(matrix)):
        if i != user_idx and not np.isnan(matrix[i, item_idx]):
            sim = pearson_similarity(target_user, matrix[i])
            if sim > 0:  # Chỉ xét users có correlation dương
                similarities.append(sim)
                ratings.append(matrix[i, item_idx])
    
    if len(similarities) == 0:
        return np.mean(target_user[~np.isnan(target_user)])  # Fallback
    
    # Weighted average
    weighted_sum = sum(sim * rating for sim, rating in zip(similarities, ratings))
    sim_sum = sum(similarities)
    
    return weighted_sum / sim_sum if sim_sum != 0 else np.mean(ratings)

# Dự đoán r_U1,I3 (user 0, item 2)
pred_u1_i3 = predict_rating_pearson(0, 3, utility_matrix)
print(f"\n1. Dự đoán bằng Pearson correlation:")
print(f"r_U1,I3 = {pred_u1_i3:.3f}")

# Dự đoán r_U4,I1 (user 3, item 0)  
pred_u4_i1 = predict_rating_pearson(4, 2, utility_matrix)
print(f"r_U4,I1 = {pred_u4_i1:.3f}")

# Hiển thị ma trận similarity Pearson
print(f"\nMa trận tương đồng Pearson:")
pearson_sim_matrix = np.zeros((8, 8))
for i in range(8):
    for j in range(8):
        if i != j:
            pearson_sim_matrix[i, j] = pearson_similarity(utility_matrix[i], utility_matrix[j])
        else:
            pearson_sim_matrix[i, j] = 1.0

pearson_df = pd.DataFrame(pearson_sim_matrix, index=users, columns=users)
display(pearson_df.round(3))

Utility Matrix:


,I1,I2,I3,I4,I5
U1,4.0,3.0,4.0,NaN,5.0
U2,2.0,3.0,1.0,5.0,3.0
U3,5.0,4.0,3.0,4.0,2.0
U4,4.0,2.0,2.0,5.0,4.0
U5,3.0,4.0,NaN,3.0,5.0
U6,5.0,3.0,5.0,5.0,1.0
U7,5.0,5.0,5.0,4.0,5.0
U8,1.0,5.0,2.0,2.0,4.0



1. Dự đoán bằng Pearson correlation:
r_U1,I3 = 4.172
r_U4,I1 = 3.438

Ma trận tương đồng Pearson:


,U1,U2,U3,U4,U5,U6,U7,U8
U1,1.000,0.000,-0.632,0.707,0.500,-0.426,0.000,-0.224
U2,0.000,1.000,0.089,0.678,-0.208,-0.113,-0.829,0.185
U3,-0.632,0.089,1.000,0.131,-0.899,0.686,-0.196,-0.454
U4,0.707,0.678,0.131,1.000,-0.346,0.042,-0.667,-0.408
U5,0.500,-0.208,-0.899,-0.346,1.000,-1.000,0.522,0.763
U6,-0.426,-0.113,0.686,0.042,-1.000,1.000,-0.375,-0.782
U7,0.000,-0.829,-0.196,-0.667,0.522,-0.375,1.000,0.272
U8,-0.224,0.185,-0.454,-0.408,0.763,-0.782,0.272,1.000


### 3.2

In [76]:
# 2. Chuẩn hóa center và dùng cosine similarity

# Tính mean của mỗi user (bỏ qua NaN)
user_means = np.nanmean(utility_matrix, axis=1)
print(f"User means: {user_means}")

# Tạo centered matrix
centered_matrix = utility_matrix.copy()
for i in range(len(utility_matrix)):
    for j in range(len(utility_matrix[i])):
        if not np.isnan(utility_matrix[i, j]):
            centered_matrix[i, j] = utility_matrix[i, j] - user_means[i]

print(f"\nCentered Matrix:")
centered_df = pd.DataFrame(centered_matrix, index=users, columns=items)
display(centered_df.round(3))

def predict_rating_cosine_centered(user_idx, item_idx, original_matrix, centered_matrix, user_means):
    """Dự đoán rating bằng cosine similarity trên centered data"""
    target_user_centered = centered_matrix[user_idx]
    similarities = []
    centered_ratings = []
    
    for i in range(len(centered_matrix)):
        if i != user_idx and not np.isnan(original_matrix[i, item_idx]):
            # Tính cosine similarity giữa 2 vectors (bỏ qua NaN)
            user1 = centered_matrix[user_idx]
            user2 = centered_matrix[i]
            
            # Tìm indices không phải NaN cho cả 2 users
            valid_indices = ~(np.isnan(user1) | np.isnan(user2))
            
            if np.sum(valid_indices) > 0:
                user1_valid = user1[valid_indices].reshape(1, -1)
                user2_valid = user2[valid_indices].reshape(1, -1)
                
                # Tính cosine similarity
                sim = cosine_similarity(user1_valid, user2_valid)[0, 0]
                
                if not np.isnan(sim) and sim > 0:
                    similarities.append(sim)
                    centered_ratings.append(centered_matrix[i, item_idx])
    
    if len(similarities) == 0:
        return user_means[user_idx]  # Fallback to user mean
    
    # Weighted average của centered ratings
    weighted_sum = sum(sim * rating for sim, rating in zip(similarities, centered_ratings))
    sim_sum = sum(similarities)
    
    centered_prediction = weighted_sum / sim_sum if sim_sum != 0 else np.mean(centered_ratings)
    
    # Thêm lại user mean
    return centered_prediction + user_means[user_idx]

pred_u1_i3_cosine = predict_rating_cosine_centered(0, 3, utility_matrix, centered_matrix, user_means)
print(f"\n2. Dự đoán bằng Cosine similarity (centered):")
print(f"r_U1,I3 = {pred_u1_i3_cosine:.3f}")


pred_u4_i1_cosine = predict_rating_cosine_centered(4, 2, utility_matrix, centered_matrix, user_means)
print(f"r_U4,I1 = {pred_u4_i1_cosine:.3f}")

# Ma trận cosine similarity cho centered data
cosine_sim_matrix = np.zeros((8, 8))
for i in range(8):
    for j in range(8):
        if i != j:
            user1 = centered_matrix[i]
            user2 = centered_matrix[j]
            valid_indices = ~(np.isnan(user1) | np.isnan(user2))
            
            if np.sum(valid_indices) > 0:
                user1_valid = user1[valid_indices].reshape(1, -1)
                user2_valid = user2[valid_indices].reshape(1, -1)
                sim = cosine_similarity(user1_valid, user2_valid)[0, 0]
                cosine_sim_matrix[i, j] = sim if not np.isnan(sim) else 0
            else:
                cosine_sim_matrix[i, j] = 0
        else:
            cosine_sim_matrix[i, j] = 1.0

print(f"\nMa trận tương đồng Cosine (centered data):")
cosine_df = pd.DataFrame(cosine_sim_matrix, index=users, columns=users)
display(cosine_df.round(3))

User means: [4.   2.8  3.6  3.4  3.75 3.8  4.8  2.8 ]

Centered Matrix:


,I1,I2,I3,I4,I5
U1,0.00,-1.00,0.0,NaN,1.00
U2,-0.80,0.20,-1.8,2.20,0.20
U3,1.40,0.40,-0.6,0.40,-1.60
U4,0.60,-1.40,-1.4,1.60,0.60
U5,-0.75,0.25,NaN,-0.75,1.25
U6,1.20,-0.80,1.2,1.20,-2.80
U7,0.20,0.20,0.2,-0.80,0.20
U8,-1.80,2.20,-0.8,-0.80,1.20



2. Dự đoán bằng Cosine similarity (centered):
r_U1,I3 = 4.610
r_U4,I1 = 3.464

Ma trận tương đồng Cosine (centered data):


,U1,U2,U3,U4,U5,U6,U7,U8
U1,1.000,0.000,-0.630,0.657,0.478,-0.420,0.000,-0.222
U2,0.000,1.000,0.089,0.678,-0.192,-0.113,-0.829,0.185
U3,-0.630,0.089,1.000,0.131,-0.891,0.686,-0.196,-0.454
U4,0.657,0.678,0.131,1.000,-0.329,0.042,-0.667,-0.408
U5,0.478,-0.192,-0.891,-0.329,1.000,-0.984,0.519,0.757
U6,-0.420,-0.113,0.686,0.042,-0.984,1.000,-0.375,-0.782
U7,0.000,-0.829,-0.196,-0.667,0.519,-0.375,1.000,0.272
U8,-0.222,0.185,-0.454,-0.408,0.757,-0.782,0.272,1.000


**Bài 4.** (*2.5 điểm*) Sử dụng lại dataset đã cho ở **Bài 1** và chọn một thuật toán *collaborative filtering* đã chuẩn bị sẵn, trong đó ở ý tưởng kNN, ta chọn $k=5$.

1) Lọc và giữ lại các user đã rate cho ít nhất $5$ phim. Từ đó in ra ma trận tương đồng giữa các user đó dựa trên độ tương đồng phù hợp.
2) Hãy nêu một số ưu điểm-nhược điểm cơ bản khi so sánh giữa khuyến nghị theo *user-based* và *item-based*.
3) Duyệt theo từng user và cho biết sẽ khuyến nghị các item nào cho họ.
4) Duyệt theo từng item và cho biết sẽ khuyến nghị item này cho các user nào.

### 4.1

In [82]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('data.csv', names=['userid', 'songid', 'rating'])


# 1) Tạo utility matrix (user-item matrix)
# Xử lý duplicate entries bằng cách lấy giá trị trung bình
usser_fillter = df.groupby('userid').agg(num_ratings=('rating', 'count'),
                                      avg_rating =('rating', 'mean')).reset_index()

usser_fillter = usser_fillter[usser_fillter['num_ratings'] >= 5].copy()
usser_fillter

,userid,num_ratings,avg_rating
1,3,9,2.333333
3,5,7,3.142857
5,7,6,2.833333
8,10,8,3.125000
9,11,9,3.555556
...,...,...,...
351,384,6,2.500000
352,385,7,3.285714
358,391,6,2.833333
359,392,7,2.714286


In [87]:
df_clean = df[df["userid"].isin(usser_fillter["userid"])]
utility_matrix = df_clean.pivot(index='userid', columns='songid', values='rating')
print(f"\nUtility Matrix shape: {utility_matrix.shape}")
display(utility_matrix.head())

# Điền NaN bằng 0 cho việc tính toán similarity
utility_filled = utility_matrix.fillna(0)

# 1) In ra ma trận tương đồng giữa các user và giữa các item

# Ma trận tương đồng User-User (cosine similarity)
user_similarity = cosine_similarity(utility_filled)
user_sim_df = pd.DataFrame(user_similarity, 
                          index=utility_matrix.index, 
                          columns=utility_matrix.index)

print(f"\nMa trận tương đồng User-User:")
display(user_sim_df.round(3))


Utility Matrix shape: (183, 25)


songid,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
userid,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,3.0,NaN,...,NaN,1.0,3.0,NaN,NaN,2.0,NaN,NaN,NaN,4.0
5,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,1.0
7,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,4.0,...,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,...,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,5.0,NaN
11,NaN,NaN,NaN,NaN,3.0,3.0,4.0,NaN,5.0,NaN,...,5.0,4.0,NaN,NaN,NaN,5.0,NaN,1.0,NaN,NaN



Ma trận tương đồng User-User:


userid,3,5,7,10,11,14,15,16,18,19,...,377,379,380,381,382,384,385,391,392,396
userid,,,,,,,,,,,,,,,,,,,,,
3,1.000,0.326,0.284,0.319,0.488,0.209,0.317,0.536,0.642,0.305,...,0.441,0.207,0.034,0.322,0.312,0.303,0.220,0.345,0.300,0.148
5,0.326,1.000,0.000,0.422,0.255,0.122,0.331,0.289,0.219,0.295,...,0.108,0.140,0.202,0.054,0.180,0.144,0.023,0.291,0.493,0.277
7,0.284,0.000,1.000,0.000,0.265,0.351,0.287,0.326,0.273,0.000,...,0.232,0.066,0.000,0.102,0.000,0.522,0.065,0.328,0.159,0.063
10,0.319,0.422,0.000,1.000,0.377,0.153,0.155,0.059,0.422,0.437,...,0.193,0.240,0.311,0.238,0.067,0.240,0.065,0.246,0.158,0.312
11,0.488,0.255,0.265,0.377,1.000,0.489,0.340,0.365,0.172,0.048,...,0.462,0.338,0.000,0.256,0.414,0.464,0.236,0.251,0.331,0.163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,0.303,0.144,0.522,0.240,0.464,0.502,0.260,0.241,0.265,0.063,...,0.301,0.268,0.000,0.070,0.046,1.000,0.000,0.074,0.216,0.124
385,0.220,0.023,0.065,0.065,0.236,0.000,0.071,0.187,0.043,0.000,...,0.364,0.324,0.295,0.269,0.089,0.000,1.000,0.256,0.300,0.193
391,0.345,0.291,0.328,0.246,0.251,0.000,0.119,0.293,0.107,0.214,...,0.114,0.176,0.286,0.338,0.000,0.074,0.256,1.000,0.541,0.048


In [88]:
# 2) User-based Collaborative Filtering

def predict_user_based_rating(user_id, item_id, utility_matrix, user_similarity, k=5):
    """Dự đoán rating cho user và item dựa trên user-based CF"""
    if item_id not in utility_matrix.columns or user_id not in utility_matrix.index:
        return utility_matrix.mean().mean()  # Global average
    
    # Lấy similarity của user với tất cả users khác
    user_similarities = user_similarity.loc[user_id].drop(user_id)
    
    # Lọc ra những users đã rating item này
    item_ratings = utility_matrix[item_id].dropna()
    if user_id in item_ratings.index:
        item_ratings = item_ratings.drop(user_id)
    
    if len(item_ratings) == 0:
        return utility_matrix.loc[user_id].mean()  # User average
    
    # Lấy top k users tương đồng nhất đã rating item này
    common_users = item_ratings.index.intersection(user_similarities.index)
    if len(common_users) == 0:
        return utility_matrix.loc[user_id].mean()
    
    top_similar_users = user_similarities[common_users].nlargest(k)
    
    if top_similar_users.sum() == 0:
        return utility_matrix.loc[user_id].mean()
    
    # Tính weighted average
    weighted_sum = sum(sim * item_ratings[similar_user] 
                      for similar_user, sim in top_similar_users.items())
    similarity_sum = top_similar_users.sum()
    
    return weighted_sum / similarity_sum if similarity_sum != 0 else utility_matrix.loc[user_id].mean()

# 2) Duyệt theo từng user và khuyến nghị items
def recommend_items_for_user(user_id, utility_matrix, user_similarity, n_recommendations=5):
    """Khuyến nghị items cho một user cụ thể"""
    user_ratings = utility_matrix.loc[user_id]
    unrated_items = user_ratings[user_ratings.isna()].index
    
    if len(unrated_items) == 0:
        return pd.Series(dtype=float)
    
    predictions = {}
    for item_id in unrated_items:
        pred_rating = predict_user_based_rating(user_id, item_id, utility_matrix, user_sim_df)
        predictions[item_id] = pred_rating
    
    recommendations = pd.Series(predictions).sort_values(ascending=False)
    return recommendations.head(n_recommendations)

print("2) USER-BASED COLLABORATIVE FILTERING")
print("Khuyến nghị cho từng user (Top 5 items):")

# Chọn một vài users để demo
sample_users = utility_matrix.index[:5]
user_recommendations = {}

for user_id in sample_users:
    recommendations = recommend_items_for_user(user_id, utility_matrix, user_sim_df)
    user_recommendations[user_id] = recommendations
    print(f"\nUser {user_id}:")
    if len(recommendations) > 0:
        for item_id, pred_rating in recommendations.items():
            print(f"  Song {item_id}: {pred_rating:.3f}")
    else:
        print("  Không có item nào để khuyến nghị")

2) USER-BASED COLLABORATIVE FILTERING
Khuyến nghị cho từng user (Top 5 items):

User 3:
  Song 16: 4.157
  Song 1: 3.335
  Song 22: 3.186
  Song 7: 3.173
  Song 11: 3.048

User 5:
  Song 1: 3.876
  Song 22: 3.783
  Song 10: 3.133
  Song 18: 3.057
  Song 24: 2.978

User 7:
  Song 24: 3.604
  Song 25: 3.261
  Song 3: 3.162
  Song 14: 3.115
  Song 20: 3.109

User 10:
  Song 14: 4.170
  Song 3: 3.735
  Song 7: 3.721
  Song 18: 3.435
  Song 25: 3.407

User 11:
  Song 4: 4.026
  Song 22: 3.619
  Song 20: 3.328
  Song 10: 3.299
  Song 24: 3.238


In [89]:
# 3) Item-based Collaborative Filtering

def predict_item_based_rating(user_id, item_id, utility_matrix, item_similarity, k=5):
    """Dự đoán rating cho user và item dựa trên item-based CF"""
    if item_id not in utility_matrix.columns or user_id not in utility_matrix.index:
        return utility_matrix.mean().mean()
    
    # Lấy những items mà user đã rating
    user_ratings = utility_matrix.loc[user_id].dropna()
    if item_id in user_ratings.index:
        user_ratings = user_ratings.drop(item_id)
    
    if len(user_ratings) == 0:
        return utility_matrix[item_id].mean()  # Item average
    
    # Lấy similarity của item với những items mà user đã rating
    item_similarities = item_similarity.loc[item_id, user_ratings.index]
    
    # Lấy top k items tương đồng nhất
    top_similar_items = item_similarities.nlargest(k)
    
    if top_similar_items.sum() == 0:
        return utility_matrix[item_id].mean()
    
    # Tính weighted average
    weighted_sum = sum(sim * user_ratings[similar_item] 
                      for similar_item, sim in top_similar_items.items())
    similarity_sum = top_similar_items.sum()
    
    return weighted_sum / similarity_sum if similarity_sum != 0 else utility_matrix[item_id].mean()

def recommend_users_for_item(item_id, utility_matrix, item_similarity, n_recommendations=5):
    """Khuyến nghị item cho những users chưa rating"""
    item_ratings = utility_matrix[item_id]
    unrated_users = item_ratings[item_ratings.isna()].index
    
    if len(unrated_users) == 0:
        return pd.Series(dtype=float)
    
    predictions = {}
    for user_id in unrated_users:
        pred_rating = predict_item_based_rating(user_id, item_id, utility_matrix, item_sim_df)
        predictions[user_id] = pred_rating
    
    recommendations = pd.Series(predictions).sort_values(ascending=False)
    return recommendations.head(n_recommendations)

print("\n3) ITEM-BASED COLLABORATIVE FILTERING")
print("Khuyến nghị từng item cho users (Top 5 users):")

# Chọn một vài items để demo
sample_items = utility_matrix.columns[:5]
item_recommendations = {}

for item_id in sample_items:
    recommendations = recommend_users_for_item(item_id, utility_matrix, item_sim_df)
    item_recommendations[item_id] = recommendations
    print(f"\nSong {item_id}:")
    if len(recommendations) > 0:
        for user_id, pred_rating in recommendations.items():
            print(f"  User {user_id}: {pred_rating:.3f}")
    else:
        print("  Không có user nào để khuyến nghị")


3) ITEM-BASED COLLABORATIVE FILTERING
Khuyến nghị từng item cho users (Top 5 users):

Song 1:
  User 114: 4.267
  User 112: 4.257
  User 33: 4.191
  User 303: 4.145
  User 104: 4.092

Song 2:
  User 307: 4.410
  User 11: 4.397
  User 373: 4.335
  User 114: 4.307
  User 379: 4.295

Song 3:
  User 208: 4.386
  User 373: 4.307
  User 307: 4.257
  User 303: 4.250
  User 53: 4.164

Song 4:
  User 114: 4.287
  User 31: 4.278
  User 104: 4.205
  User 276: 4.123
  User 53: 4.111

Song 5:
  User 31: 4.408
  User 373: 4.259
  User 114: 4.256
  User 112: 4.128
  User 214: 4.099


### 4.2
Ưu nhược điểm:
   - User-based CF:
     + Phù hợp khi có nhiều users, ít items
     + Có thể giải thích được (users tương đồng)
     - Không ổn định khi sở thích user thay đổi
     - Khó mở rộng với số lượng users lớn

  - Item-based CF:
     + Ổn định hơn (đặc tính items ít thay đổi)
     + Dễ tính toán trước và lưu trữ
     + Phù hợp khi có nhiều items, ít users
     - Khó giải thích cho user
     - Có thể dẫn đến filter bubble

**Bài 5.** (*0.5 điểm*) Trong HAI chọn MỘT.

1) Dựa theo **Bài 1**, nếu có một user đã nghe $3$ bài hát nào đó trong $25$ bài hát đã cho thì muốn khuyến nghị top $5$ bài hát mà user này có thể muốn nghe, anh/chị sẽ làm thế nào? Chỉ mô tả ý tưởng dựa trên cách làm ở ý **1.2)** ở trên, không cần code chi tiết.
2) Dựa theo **Bài 4**, về lý thuyết thì hai ma trận mô tả user-item trong kết quả ở **4.3)** và **4.4)** có quan hệ gì với nhau?

In [ ]:
#ANSWER HERE

5.1
- Tìm tất cả các user đã nghe cả 3 bài hát
- Tìm tất cả các cặp bài hát mà trong đó gồm 1 trong 3 bài đã cho và bài còn lại là bài người dùng chưa nghe
- Đến tần suất các cặp xuất hiện
- Chọn ra 5 cặp xuất hiện nhiều nhất và đề xuất bài còn lại trong cặp mà người dùng chưa nghe